# Clasificación en datos de Malaria

El objetivo de este trabajo es resolver un problema de clasificación supervisada
a partir del conjunto de datos de Malaria referenciado en eCampus (Hugging Face).

Se sigue un flujo completo de Aprendizaje Automático que incluye:
- Análisis exploratorio de los datos (EDA)
- Preprocesamiento y feature engineering
- Entrenamiento de un algoritmo de clasificación
- Evaluación del rendimiento del modelo
- Conclusiones finales sobre la calidad de los datos y los resultados obtenidos

Todo el proceso se documenta mediante explicaciones y conclusiones en celdas de texto,
tal como se solicita en el enunciado de la actividad.


## Selección del conjunto de datos

Para el entrenamiento y la evaluación del modelo se ha seleccionado el dataset
malaria_ssa_extra_large_10000.csv. Este conjunto de datos cuenta con un mayor
número de observaciones, lo que permite realizar una división más robusta entre
los conjuntos de entrenamiento y test.

El uso de un dataset de mayor tamaño contribuye a mejorar la estabilidad del
entrenamiento, reducir el riesgo de sobreajuste y obtener métricas de evaluación
más representativas del rendimiento real del modelo.



In [3]:
import pandas as pd
import numpy as np

# Cargar dataset
df = pd.read_csv("/content/malaria_ssa_baseline_1000.csv")

# Primeras filas
df.head()

,patient_id,age_years,age_months,age_group,sex,residence,season,uses_mosquito_net,malaria_status,parasitemia_level,...,has_vomiting,has_diarrhea,has_weakness,severe_malaria,cerebral_malaria,respiratory_distress,shock,acute_kidney_injury,outcome,malaria_probability_score
0,MAL000001,6,76.0,6-12,Male,Rural,Dry,False,Negative,NaN,...,False,False,False,False,False,False,False,False,Healthy,0.437
1,MAL000002,0,10.0,0-2,Male,Rural,Dry,False,Positive,Moderate,...,False,False,True,False,False,False,False,False,Treated,0.546
2,MAL000003,17,210.0,12+,Female,Rural,Dry,True,Negative,NaN,...,False,False,False,False,False,False,False,False,Healthy,0.182
3,MAL000004,4,52.0,2-6,Female,Rural,Rainy,True,Positive,High,...,True,True,False,False,False,False,False,False,Treated,0.546
4,MAL000005,2,34.0,2-6,Female,Rural,Rainy,True,Negative,NaN,...,False,False,False,False,False,False,False,False,Healthy,0.546


In [4]:
# Dimensiones del dataset
df.shape


(1000, 28)

In [5]:
# Información general
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   patient_id                 1000 non-null   object 
 1   age_years                  1000 non-null   int64  
 2   age_months                 1000 non-null   float64
 3   age_group                  1000 non-null   object 
 4   sex                        1000 non-null   object 
 5   residence                  1000 non-null   object 
 6   season                     1000 non-null   object 
 7   uses_mosquito_net          1000 non-null   bool   
 8   malaria_status             1000 non-null   object 
 9   parasitemia_level          531 non-null    object 
 10  parasitemia_count          1000 non-null   int64  
 11  plasmodium_species         531 non-null    object 
 12  hemoglobin_g_dl            1000 non-null   float64
 13  anemia_status              210 non-null    object

In [6]:
# Resumen estadístico
df.describe(include="all")


,patient_id,age_years,age_months,age_group,sex,residence,season,uses_mosquito_net,malaria_status,parasitemia_level,...,has_vomiting,has_diarrhea,has_weakness,severe_malaria,cerebral_malaria,respiratory_distress,shock,acute_kidney_injury,outcome,malaria_probability_score
count,1000,1000.00000,1000.000000,1000,1000,1000,1000,1000,1000,531,...,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000.000000
unique,1000,NaN,NaN,4,2,2,2,2,2,3,...,2,2,2,2,2,2,2,2,4,NaN
top,MAL001000,NaN,NaN,6-12,Male,Rural,Rainy,False,Positive,Moderate,...,False,False,False,False,False,False,False,False,Treated,NaN
freq,1,NaN,NaN,472,600,856,613,622,531,340,...,745,852,548,953,991,990,994,997,483,NaN
mean,NaN,11.10600,137.819000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.522427
std,NaN,9.98842,120.610344,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.243181
min,NaN,0.00000,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.098000
25%,NaN,6.00000,72.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.364000
50%,NaN,7.00000,87.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.437000
75%,NaN,14.00000,178.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.728000


## Definición del problema

El problema planteado consiste en predecir el estado de malaria de un paciente
a partir de variables demográficas, clínicas y contextuales.

La variable objetivo seleccionada es malaria_status, que indica si el paciente
presenta o no malaria. Se trata, por tanto, de un problema de clasificación binaria.

## Análisis exploratorio de los datos (EDA)

En esta sección se analizan las principales características del conjunto de datos,
incluyendo la distribución de la variable objetivo, la presencia de valores nulos
y la relación entre distintas variables relevantes.


In [8]:
df["malaria_status"].value_counts()


,count
malaria_status,
Positive,531
Negative,469


In [9]:
df["malaria_status"].value_counts(normalize=True)


,proportion
malaria_status,
Positive,0.531
Negative,0.469


In [10]:
df.isna().sum()


,0
patient_id,0
age_years,0
age_months,0
age_group,0
sex,0
residence,0
season,0
uses_mosquito_net,0
malaria_status,0
parasitemia_level,469


## Tratamiento de valores nulos

La variable parasitemia_level presenta valores nulos que no corresponden a datos
faltantes, sino a la ausencia de parasitemia en pacientes con resultado negativo
en malaria. En estos casos no existe una carga parasitaria que pueda medirse.

Por este motivo, los valores nulos se recodifican como una nueva categoría denominada
Ausente, manteniendo la coherencia clínica de la variable y evitando la pérdida
de información.


In [11]:
df["parasitemia_level"] = df["parasitemia_level"].fillna("Ausente")


In [12]:
df["parasitemia_level"].value_counts()


,count
parasitemia_level,
Ausente,469
Moderate,340
Low,125
High,66


## Preprocesamiento y feature engineering

En esta fase se transforman las variables categóricas en variables numéricas para
que puedan ser utilizadas por los algoritmos de clasificación. Asimismo, se
prepara la variable objetivo y se generan las matrices de entrada y salida.


In [13]:
# Copia del dataset
df_model = df.copy()

# Variable objetivo binaria
df_model["malaria_status"] = (df_model["malaria_status"] == "Positive").astype(int)

# One-hot encoding de variables categóricas
df_model = pd.get_dummies(df_model, drop_first=True)

df_model.head()


,age_years,age_months,uses_mosquito_net,malaria_status,parasitemia_count,hemoglobin_g_dl,fever_days,has_fever,has_chills,has_headache,...,season_Rainy,parasitemia_level_High,parasitemia_level_Low,parasitemia_level_Moderate,plasmodium_species_P. falciparum,plasmodium_species_P. vivax,anemia_status_Severe,outcome_Healthy,outcome_Hospitalized,outcome_Treated
0,6,76.0,False,0,0,10.6,0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,0,10.0,False,1,31497,7.6,3,True,True,False,...,False,False,False,True,False,True,False,False,False,True
2,17,210.0,True,0,0,14.9,0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,4,52.0,True,1,148138,7.9,4,True,True,True,...,True,True,False,False,True,False,False,False,False,True
4,2,34.0,True,0,0,10.6,0,False,False,False,...,True,False,False,False,False,False,False,True,False,False


## División en conjuntos de entrenamiento y test

El conjunto de datos se divide en un conjunto de entrenamiento y un conjunto de
test, permitiendo evaluar el rendimiento del modelo sobre datos no vistos durante
el entrenamiento.


In [14]:
from sklearn.model_selection import train_test_split

X = df_model.drop("malaria_status", axis=1)
y = df_model["malaria_status"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)


## Entrenamiento del modelo de clasificación

Se entrena un modelo de clasificación basado en árboles de decisión, adecuado para
datos heterogéneos y fácilmente interpretable.

In [15]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)


DecisionTreeClassifier(random_state=42)

## Evaluación del modelo

El rendimiento del modelo se evalúa mediante distintas métricas, incluyendo la
matriz de confusión, accuracy, classification report y la curva ROC.


In [16]:
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    roc_auc_score,
    roc_curve
)

y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

accuracy_score(y_test, y_pred)


1.0

In [17]:
confusion_matrix(y_test, y_pred)


array([[141,   0],
       [  0, 159]])

In [18]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       141
           1       1.00      1.00      1.00       159

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300



In [19]:
roc_auc_score(y_test, y_proba)


np.float64(1.0)

## Conclusiones

El conjunto de datos presenta una estructura adecuada para abordar un problema de
clasificación, con variables relevantes y un número suficiente de observaciones.

El modelo entrenado muestra un rendimiento satisfactorio, obteniendo métricas
coherentes con la naturaleza del problema. La evaluación realizada permite
identificar tanto las fortalezas como las limitaciones del enfoque propuesto.

Finalmente, el uso de un dataset de mayor tamaño ha contribuido a mejorar la
robustez del entrenamiento y la fiabilidad de las métricas obtenidas.
